In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Data.csv to Data.csv


In [ ]:
import pandas as pd
df=pd.read_csv('Data.csv')

In [ ]:
df=df[['Tweet','ADR_label']]

In [ ]:
#Data cleaning and preprocessing
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(df)):
    review = re.sub('[^a-zA-Z]', ' ', df['Tweet'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
df.head()

,Tweet,ADR_label
0,Intravenous azithromycin-induced ototoxicity.,1
1,"Immobilization, while Paget's bone disease was...",1
2,Unaccountable severe hypercalcemia in a patien...,1
3,METHODS: We report two cases of pseudoporphyri...,1
4,METHODS: We report two cases of pseudoporphyri...,1


In [ ]:
# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500)
X = cv.fit_transform(corpus).toarray()

In [ ]:
# Creating the TFIDF model
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features=1500)
X = tv.fit_transform(corpus).toarray()

In [ ]:
X[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
!pip show gensim

Name: gensim
Version: 3.6.0
Summary: Python framework for fast Vector Space Modelling
Home-page: http://radimrehurek.com/gensim
Author: Radim Rehurek
Author-email: me@radimrehurek.com
License: LGPLv2.1
Location: /usr/local/lib/python3.7/dist-packages
Requires: numpy, six, smart-open, scipy
Required-by: 


Word To Vec 
Pretrained

In [ ]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
corpus = []
for i in range(0, len(df)):
    review = re.sub('[^a-zA-Z]', ' ', df['Tweet'][i])
    review = review.lower()
    review = review.split()
    
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
from nltk import sent_tokenize
from gensim.utils import simple_preprocess
words=[]
for sent in corpus:
    sent_token=sent_tokenize(sent)
    for sent in sent_token:
        words.append(simple_preprocess(sent))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Word To Vec from Scratch

In [ ]:
words[0]

['intravenous', 'azithromycin', 'induced', 'ototoxicity']

In [ ]:
### Lets train Word2vec from scratch
import gensim
model=gensim.models.Word2Vec(words,window=5,min_count=2)

In [ ]:
model.corpus_count

23516

In [ ]:
model.epochs

5

In [ ]:
def avg_word2vec(doc):
    # remove out-of-vocabulary words
    #sent = [word for word in doc if word in model.wv.index_to_key]
    #print(sent)
    
    return np.mean([model.wv[word] for word in doc if word in model.wv.index2word],axis=0)
                #or [np.zeros(len(model.wv.index_to_key))], axis=0)
        
    
    

In [ ]:
from tqdm import tqdm

In [ ]:
#apply for the entire sentences
import numpy as np
X=[]
for i in tqdm(range(len(words))):
    X.append(avg_word2vec(words[i]))


 37%|███▋      | 8681/23516 [00:03<00:06, 2203.13it/s]/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 23516/23516 [00:15<00:00, 1476.55it/s]


Transformers:

1. Use Pretrained model directly as a classifier
2. Transformer model to extract embedding and use it as input to another classifier.
3. Fine-tuning a Pretrained transformer model on custom config and dataset.

In [ ]:

Build
ANN
RNN
LSTM
Bi-LSTM
BERT
GPT
Roberta
Albert
Distilbert

## Artificial Network

In [ ]:
X.shape

(23516, 1500)

In [ ]:
y=df['ADR_label']

In [ ]:
## Divide the dataset into Train and Test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [ ]:
import tensorflow
y_train=tensorflow.keras.utils.to_categorical(y_train, num_classes=2)
y_test=tensorflow.keras.utils.to_categorical(y_test, num_classes=2)


In [ ]:
y=tensorflow.keras.utils.to_categorical(y, num_classes=2)

In [ ]:


# X_train = np.array(X_train).reshape((X_train.shape[0], X_train.shape[1], 1))
# X_test = np.array(X_test).reshape((X_test.shape[0], X_test.shape[1], 1))
# X_val = np.array(X_val).reshape((X_val.shape[0], X_val.shape[1], 1))

In [ ]:
print(X_train.shape, X_test.shape)

(15755, 1500) (7761, 1500)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import regularizers, optimizers

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import regularizers, optimizers

def train_and_test_loop(iterations, lr, Lambda, verb=True):

    ## hyperparameters
    iterations = iterations
    learning_rate = lr
    hidden_nodes = 512
    output_nodes = 2
        
    model = Sequential()
    model.add(Dense(hidden_nodes, input_shape=(2500,), activation='relu'))
    model.add(Dense(hidden_nodes, activation='relu'))
    model.add(Dense(output_nodes, activation='softmax', kernel_regularizer=regularizers.l2(Lambda)))
    
    sgd = optimizers.SGD(lr=learning_rate, decay=1e-6, momentum=0.9)
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    
    # Fit the model
    model.fit(X_train, y_train, epochs=iterations, batch_size=32, verbose= 1)
    score_train=model.evaluate(X_train,y_train)
    score_val=model.evaluate(X_test,y_test)

    return score_train, score_val

In [ ]:
def train_and_test_loop1(iterations, lr, Lambda, verb=True):

    ## hyperparameters
    iterations = iterations
    learning_rate = lr
    hidden_nodes = 256
    output_nodes = 2

    model = Sequential()
    model.add(Dense(hidden_nodes, input_shape=(2500,), activation='relu'))
    model.add(Dense(hidden_nodes, activation='relu'))
    model.add(Dense(output_nodes, activation='softmax', kernel_regularizer=regularizers.l2(Lambda)))
    
    sgd = optimizers.SGD(lr=learning_rate, decay=1e-6, momentum=0.9)
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    
    # Fit the model
    model.fit(X_train_subset, y_train_subset, epochs=iterations, batch_size=32, verbose= 1)
    #score = model.evaluate(X_test, y_test, verbose=0)
    
    # return score

(7761, 2500)

### Next steps 
- Double Check that the loss is reasonable
- Disable the regularization (Lambda = 0)

In [ ]:
lr = 0.00001
Lambda = 0
train_and_test_loop(1, lr, Lambda)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


493/493 [==============================] - 7s 13ms/step - loss: 0.6939 - accuracy: 0.4763


### Question
- Is the loss range correct? What about accuracy, does it make sense for an untrained network

### Answer
- Absolutely! There are 2 output classes and the model is correctly predicting 1 up on 2 times (1/2 = 0.5% approx) as it is untrained.

### Now, lets crank up the Lambda(Regularization)and check what it does to our loss function.




In [ ]:
lr = 0.00001
Lambda = 1e3
train_and_test_loop(1, lr, Lambda)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


493/493 [==============================] - 7s 14ms/step - loss: 61.3253 - accuracy: 0.6962


### Now, lets overfit to a small subset of our dataset, in this case 20 images, to ensure our model architecture is good


In [ ]:
X_train_subset = X_train[0:20]

y_train_subset = y_train[0:20]

### Tip: Make sure that you can overfit very small portion of the training data


So, set a small learning rate and turn regularization off

In the code below:
- Take the first 20 examples from MNIST
- turn off regularization(reg=0.0)
- use simple vanilla 'sgd'

Lets try and run for 500 iterations as the data set is very small

In [ ]:
lr = 0.001
Lambda = 0
train_and_test_loop1(500, lr, Lambda)

### Very small loss,  train accuracy going to 100, nice! We are successful in overfitting. The model architecture looks fine. Lets go for fine tuning it.

### Loading the original dataset again

Epoch 1/500


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


1/1 [==============================] - 0s 463ms/step - loss: 0.6902 - accuracy: 0.5500
Epoch 2/500
1/1 [==============================] - 0s 17ms/step - loss: 0.6901 - accuracy: 0.5500
Epoch 3/500
1/1 [==============================] - 0s 15ms/step - loss: 0.6900 - accuracy: 0.6000
Epoch 4/500
1/1 [==============================] - 0s 15ms/step - loss: 0.6898 - accuracy: 0.6500
Epoch 5/500
1/1 [==============================] - 0s 17ms/step - loss: 0.6896 - accuracy: 0.6500
Epoch 6/500
1/1 [==============================] - 0s 15ms/step - loss: 0.6893 - accuracy: 0.6500
Epoch 7/500
1/1 [==============================] - 0s 15ms/step - loss: 0.6889 - accuracy: 0.6500
Epoch 8/500
1/1 [==============================] - 0s 22ms/step - loss: 0.6885 - accuracy: 0.7000
Epoch 9/500
1/1 [==============================] - 0s 17ms/step - loss: 0.6881 - accuracy: 0.7000
Epoch 10/500
1/1 [==============================] - 0s 13ms/step - loss: 0.6877 - accuracy: 0.7000
Epoch 11/500
1/1 [============

In [ ]:
## Reshape Features if needed
## Normalize features if needed


In [ ]:
lr = 1e-7
Lambda = 1e-7
train_and_test_loop(20, lr, Lambda)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/20
493/493 [==============================] - 7s 14ms/step - loss: 0.6968 - accuracy: 0.3714
Epoch 2/20
493/493 [==============================] - 6s 13ms/step - loss: 0.6967 - accuracy: 0.3744
Epoch 3/20
493/493 [==============================] - 7s 13ms/step - loss: 0.6967 - accuracy: 0.3760
Epoch 4/20
493/493 [==============================] - 8s 17ms/step - loss: 0.6966 - accuracy: 0.3779
Epoch 5/20
493/493 [==============================] - 8s 16ms/step - loss: 0.6965 - accuracy: 0.3805
Epoch 6/20
493/493 [==============================] - 6s 13ms/step - loss: 0.6964 - accuracy: 0.3829
Epoch 7/20
493/493 [==============================] - 6s 13ms/step - loss: 0.6963 - accuracy: 0.3863
Epoch 8/20
493/493 [==============================] - 7s 13ms/step - loss: 0.6962 - accuracy: 0.3884
Epoch 9/20
493/493 [==============================] - 6s 13ms/step - loss: 0.6961 - accuracy: 0.3911
Epoch 10/20
493/493 [==============================] - 7s 14ms/step - loss: 0.6961 - accura

In [ ]:
X_test.shape

(2352, 2500, 1)

In [ ]:
### Loss barely changing. Learning rate is probably too low.

### Okay now lets try a (larger) learning rate 1e6. What could possibly go wrong?

# - Learning rate lr = 1e8
# - Regularization lambda = 1e-7


lr = 1e8
Lambda = 1e-7
train_and_test_loop(10, lr, Lambda)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


493/493 [==============================] - 6s 12ms/step - loss: nan - accuracy: 0.7102
Epoch 2/10
493/493 [==============================] - 6s 12ms/step - loss: nan - accuracy: 0.7105
Epoch 3/10
493/493 [==============================] - 6s 13ms/step - loss: nan - accuracy: 0.7105
Epoch 4/10
493/493 [==============================] - 6s 12ms/step - loss: nan - accuracy: 0.7105
Epoch 5/10
493/493 [==============================] - 6s 12ms/step - loss: nan - accuracy: 0.7105
Epoch 6/10
493/493 [==============================] - 6s 12ms/step - loss: nan - accuracy: 0.7105
Epoch 7/10
493/493 [==============================] - 6s 13ms/step - loss: nan - accuracy: 0.7105
Epoch 8/10
493/493 [==============================] - 6s 12ms/step - loss: nan - accuracy: 0.7105
Epoch 9/10
493/493 [==============================] - 6s 13ms/step - loss: nan - accuracy: 0.7105
Epoch 10/10
493/493 [==============================] - 7s 13ms/step - loss: nan - accuracy: 0.7105


### Hyperparameter Optimization

### Cross validation Strategy


- Do coarse -> fine cross-validation in stages

- First stage: only a few epochs to get rough idea of what params work
- Second stage: longer running time, finer search
- … (repeat as necessary)

### Tip for detecting explosions in the solver: 
- If the cost is ever > 3 * original cost, break out early


In [ ]:
### For example: Run coarse search for 10 times with different lr and Lambda values each with 100 epochs.
import math
for k in range(1,10):
    lr = math.pow(10, np.random.uniform(-5.0, 2.0))
    Lambda = math.pow(10, np.random.uniform(-7,-2))
    best_acc = train_and_test_loop(30, lr, Lambda, False)
    print("Try {0}/{1}: Best_val_acc: {2}, lr: {3}, Lambda: {4}\n".format(k, 100, best_acc, lr, Lambda))


Epoch 1/30


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


493/493 [==============================] - 8s 15ms/step - loss: 0.5867 - accuracy: 0.7156
Epoch 2/30
493/493 [==============================] - 5s 11ms/step - loss: 0.6243 - accuracy: 0.7000
Epoch 3/30
493/493 [==============================] - 6s 12ms/step - loss: 0.6173 - accuracy: 0.7063
Epoch 4/30
493/493 [==============================] - 6s 12ms/step - loss: 0.6204 - accuracy: 0.7086
Epoch 5/30
493/493 [==============================] - 6s 13ms/step - loss: 0.6180 - accuracy: 0.7087
Epoch 6/30
493/493 [==============================] - 6s 13ms/step - loss: 0.6204 - accuracy: 0.7054
Epoch 7/30
493/493 [==============================] - 6s 12ms/step - loss: 0.6258 - accuracy: 0.7034
Epoch 8/30
493/493 [==============================] - 6s 12ms/step - loss: 0.6153 - accuracy: 0.7017
Epoch 9/30
493/493 [==============================] - 6s 12ms/step - loss: 0.6162 - accuracy: 0.7073
Epoch 10/30
493/493 [==============================] - 6s 13ms/step - loss: 0.6211 - accuracy: 0.7007


In [ ]:

### As you can see from above, Case 2, 3 and 7 yields good accuracy. It is better to focus on those values for learning rate and Lambda

### Now run finer search

import math
for k in range(1,5):
    lr = math.pow(10, np.random.uniform(-4.0, -1.0))
    Lambda = math.pow(10, np.random.uniform(-4,-2))
    best_acc = train_and_test_loop1(100, lr, Lambda, False)
    print("Try {0}/{1}: Best_val_acc: {2}, lr: {3}, Lambda: {4}\n".format(k, 100, best_acc, lr, Lambda))

![alt text](https://)### Running deep with lr=0.02 and Lambda=1e-4

lr = 2e-2
Lambda = 1e-4
train_and_test_loop1(100, lr, Lambda)

In [ ]:
lr = 1e-3
Lambda = 1e-5
train_and_test_loop(50, lr, Lambda)

Epoch 1/50


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


493/493 [==============================] - 8s 16ms/step - loss: 0.6115 - accuracy: 0.7099
Epoch 2/50
493/493 [==============================] - 7s 14ms/step - loss: 0.5990 - accuracy: 0.7105
Epoch 3/50
493/493 [==============================] - 6s 13ms/step - loss: 0.5965 - accuracy: 0.7105
Epoch 4/50
493/493 [==============================] - 7s 14ms/step - loss: 0.5932 - accuracy: 0.7105
Epoch 5/50
493/493 [==============================] - 7s 15ms/step - loss: 0.5888 - accuracy: 0.7105
Epoch 6/50
493/493 [==============================] - 6s 13ms/step - loss: 0.5825 - accuracy: 0.7105
Epoch 7/50
493/493 [==============================] - 6s 13ms/step - loss: 0.5736 - accuracy: 0.7105
Epoch 8/50
493/493 [==============================] - 7s 14ms/step - loss: 0.5609 - accuracy: 0.7106
Epoch 9/50
493/493 [==============================] - 6s 12ms/step - loss: 0.5430 - accuracy: 0.7113
Epoch 10/50
493/493 [==============================] - 6s 13ms/step - loss: 0.5189 - accuracy: 0.7232


([0.17345312237739563, 0.9331005811691284],
 [0.33729493618011475, 0.8717948794364929])

RNN

In [ ]:
print(X.shape, y.shape, X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(23516, 1500) (23516, 2) (15755, 1500) (15755, 2) (7761, 1500) (7761, 2)


In [ ]:
from keras.layers import SimpleRNN, LSTM, Bidirectional

def RNN_train_and_test_loop(iterations, lr, Lambda, verb=True):

    ## hyperparameters
    iterations = iterations
    learning_rate = lr
    hidden_nodes = 512
    output_nodes = 2

    model=Sequential()
    model.add(SimpleRNN(50, input_shape=(1500,1), return_sequences=False))
    # model.add(SimpleRNN(50, input_shape=(2500,1), return_sequences=False))
    model.add(Dense(hidden_nodes, activation='relu'))
    model.add(Dense(hidden_nodes, activation='relu'))
    model.add(Dense(2, activation='softmax', kernel_regularizer=regularizers.l2(Lambda)))
    adam=optimizers.Adam(learning_rate=learning_rate)
    # sgd = optimizers.SGD(learning_rate=learning_rate, decay=1e-6, momentum=0.9)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    model.fit(X, y,  validation_split=0.33, epochs=iterations, batch_size=32, verbose= 1)
    # score_train=model.evaluate(X_train,y_train)
    # score_eval=model.evaluate(X_test,y_test)

    # return score_train,score_eval
    

In [ ]:
iterations=3
lr=1e-6
Lambda=0
RNN_train_and_test_loop(iterations, lr, Lambda)

Epoch 1/3
330/330 [==============================] - 149s 450ms/step - loss: 0.6805 - accuracy: 0.6949 - val_loss: 0.6681 - val_accuracy: 0.7038
Epoch 2/3
330/330 [==============================] - 147s 446ms/step - loss: 0.6537 - accuracy: 0.7138 - val_loss: 0.6441 - val_accuracy: 0.7038
Epoch 3/3
243/243 [==============================] - 28s 117ms/step - loss: 0.6239 - accuracy: 0.7087


([0.6230782866477966, 0.7105045914649963],
 [0.6238794326782227, 0.7086715698242188])

In [ ]:
iterations=2
lr=1e6
Lambda=1e-5
RNN_train_and_test_loop(iterations, lr, Lambda)

Epoch 1/2
493/493 [==============================] - 233s 468ms/step - loss: 2775936619617974222848.0000 - accuracy: 0.5043 - val_loss: 24724474167296.0000 - val_accuracy: 0.0000e+00
Epoch 2/2
493/493 [==============================] - 228s 462ms/step - loss: 43259179040768.0000 - accuracy: 0.5120 - val_loss: 66231373660160.0000 - val_accuracy: 0.0000e+00


In [ ]:
iterations=5
lr=1e-6
Lambda=1e-5
RNN_train_and_test_loop(iterations, lr, Lambda)

Epoch 1/5
493/493 [==============================] - 497s 1s/step - loss: 0.6908 - accuracy: 0.5597 - val_loss: 0.6697 - val_accuracy: 0.9942
Epoch 2/5
493/493 [==============================] - 479s 971ms/step - loss: 0.6885 - accuracy: 0.5666 - val_loss: 0.6453 - val_accuracy: 1.0000
Epoch 3/5
493/493 [==============================] - 475s 963ms/step - loss: 0.6850 - accuracy: 0.5670 - val_loss: 0.6194 - val_accuracy: 1.0000
Epoch 4/5
493/493 [==============================] - 486s 987ms/step - loss: 0.6829 - accuracy: 0.5670 - val_loss: 0.6027 - val_accuracy: 1.0000
Epoch 5/5
493/493 [==============================] - 487s 989ms/step - loss: 0.6815 - accuracy: 0.5670 - val_loss: 0.5909 - val_accuracy: 1.0000


LSTM, GRU, CuDNNLSTM, CuDNNGRU

In [ ]:
from keras.layers import GRU, LSTM, CuDNNGRU, CuDNNLSTM, Activation
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential

In [ ]:
def LSTM_train_and_test_loop(iterations, lr, Lambda, verb=True):

    ## hyperparameters
    iterations = iterations
    learning_rate = lr
    LSTM_Units = 100
    output_nodes = 2

    model = Sequential()
    model.add(LSTM(LSTM_Units, input_shape = (2500,1), return_sequences = True))
    model.add(LSTM(output_nodes, return_sequences = False))
    model.add(Activation('sigmoid',kernel_regularizer=regularizers.l2(Lambda)))

    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    adam=optimizers.Adam(learning_rate=learning_rate)
    # sgd = optimizers.SGD(learning_rate=learning_rate, decay=1e-6, momentum=0.9)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    model.fit(X, y,  validation_split=0.33, epochs=iterations, batch_size=32, verbose= 1)


In [ ]:
def GRU_train_and_test_loop(iterations, lr, Lambda, verb=True):

    ## hyperparameters
    iterations = iterations
    learning_rate = lr
    GRU_Units = 100
    output_nodes = 2

    model = Sequential()
    model.add(GRU(GRU_Units, input_shape = (2500,1), return_sequences = True))
    model.add(GRU(output_nodes, return_sequences = False))
    model.add(Activation('sigmoid',kernel_regularizer=regularizers.l2(Lambda)))

    adam=optimizers.Adam(learning_rate=learning_rate)
    # sgd = optimizers.SGD(learning_rate=learning_rate, decay=1e-6, momentum=0.9)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    model.fit(X, y,  validation_split=0.33, epochs=iterations, batch_size=32, verbose= 1)


In [ ]:
def CuDNNLSTM_train_and_test_loop(iterations, lr, Lambda, verb=True):

    ## hyperparameters
    iterations = iterations
    learning_rate = lr
    LSTM_Units = 100
    output_nodes = 2

    model = Sequential()
    model.add(CuDNNLSTM(LSTM_Units, input_shape = (300,1), return_sequences = True))
    model.add(CuDNNLSTM(output_nodes, return_sequences = False))
    model.add(Activation('sigmoid',kernel_regularizer=regularizers.l2(Lambda)))

    adam=optimizers.Adam(learning_rate=learning_rate)
    # sgd = optimizers.SGD(learning_rate=learning_rate, decay=1e-6, momentum=0.9)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    model.fit(X, y,  validation_split=0.33, epochs=iterations, batch_size=32, verbose= 1)


In [ ]:
def CuDNNGRU_train_and_test_loop(iterations, lr, Lambda, verb=True):

    ## hyperparameters
    iterations = iterations
    learning_rate = lr
    GRU_Units = 100
    output_nodes = 2

    model = Sequential()
    model.add(CuDNNGRU(GRU_Units, input_shape = (2500,1), return_sequences = True))
    model.add(CuDNNGRU(output_nodes, return_sequences = False))
    model.add(Activation('sigmoid',kernel_regularizer=regularizers.l2(Lambda)))

    adam=optimizers.Adam(learning_rate=learning_rate)
    # sgd = optimizers.SGD(learning_rate=learning_rate, decay=1e-6, momentum=0.9)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    model.fit(X, y,  validation_split=0.33, epochs=iterations, batch_size=32, verbose= 1)
